# Dropping a New Ball

In [ ]:
from IPython.display import Image
im="https://www.thesun.co.uk/wp-content/uploads/2025/07/hilarious-moment-trumps-golf-caddy-1012803955.jpg?strip=all&w=634"
Image(url=im, width=600, height=600)

In [ ]:
import zipfile, json, os, copy
from zipfile import ZipFile
import re, gc, os, sys, zlib, base64
from collections import Counter
from nltk.util import ngrams

p="/kaggle/input/google-code-golf-2025/code_golf_utils"
sys.path.append(p)
from code_golf_utils import *

#Since data is fized to [0,9] you can build your own compression algo
#This is just for illustration purposes
def theBirdie(s,a):
    c = Counter(list(ngrams(s, 2)))
    c=''.join(c.most_common(1)[0][0])
    s = s.replace(c,a)
    return s, c, a

def theEagle(s):
    m=[]
    alphas='abcdefghijklmnopqrstuvwxyzABCDEFGHJKLMNPQRSTUVWXYZ'
    s=str(s)
    s=s.replace(' ','')
    s=s.replace('output','O')
    s=s.replace('input','I')
    for a in alphas: #add an early stop once its no longer offsetting cost of replacement in results
        s, m1, m2 = theBirdie(s,a)
        m.append([m1,m2])
    m = str(m[::-1]).replace(' ','')
    return s,m

def theScorecard(s,m):
    sc = """def p(g):
 d='""" + str(s) + """'
 m=""" + str(m) + """
 for r in m:
  d=d.replace(r[1],r[0])
 d=eval(d)
 for k in d:
  if k['I']==g:
   g=k['O']
   return g"""
    return sc

def OneBall(s):
    foundball=[]
    hits = []
    for k in s:
        if str(k['input']) not in foundball:
            foundball.append(str(k['input']))
            hits.append(k)
    return hits
    
f =[]
scores=[]
for n in range(1,401):
    t = load_examples(n)
    s = t['train'] + t['test'] + t['arc-gen']
    s = OneBall(s)
    s,m = theEagle(s)
    c = len(s)
    if c<2300:
        s = theScorecard(s,m)
        print(n, c, len(s))
        o=open('task' + str(n).zfill(3) + '.py','w')
        o.write(s)
        o.close()
        f.append(n)
        scores.append(len(s))

print(len(f), len(f)*2500, sum([2500-x for x in scores]))
with ZipFile("submission.zip", "w", zipfile.ZIP_DEFLATED) as zipf:
    for x in f:
        zipf.write('task' + str(x).zfill(3) + '.py')

In [ ]:
#https://www.kaggle.com/code/taylorsamarel/qwen2-5-32b-arc-local-score-32-solved-script
import zipfile, json, os, copy

def check(solution, task_num, valall=False):
    task_data = load_examples(task_num)
    #print(task_num, max(1, 2500 - len(solution.encode('utf-8'))))
    try:
        namespace = {}
        exec(solution, namespace)
        if 'p' not in namespace: return False
        all_examples = task_data['train'] + task_data['test'] + task_data['arc-gen']
        examples_to_check = all_examples if valall else all_examples[:3]
        for example in examples_to_check:
            input_grid = copy.deepcopy(example['input'])
            expected = example['output']
            try:
                actual = namespace['p'](input_grid)
                if actual != expected:
                    return False
            except:
                return False
        return True
    except Exception as e:
        return False

In [ ]:
for task_num in f:
    try:
        solution = open('/kaggle/working/task' + str(task_num).zfill(3) + '.py').read()
        if check(solution, task_num, valall=True):
            print(task_num, ":)")
        else: print(task_num, ":L")
    except: pass

In [ ]:
task_num = 53
examples = load_examples(task_num)
show_examples(examples['train'] + examples['test'])

In [ ]:
%%writefile task.py
def p(g):
 d='[pF1EktAEZlAEVAEDyMCyMDGTCGTSdLtLLxlNVNDKNCKNUdPtPPSGktBGZjzQCzQDHTCHTxlMVMUyPtMPDKWCKWSzRtQRxlXVXSHRtBHRDHQCHQxlQVQDzXCzXDyMCyMDzLCzLxlBGVBGSdQtLQDGLCGLUdMtPMUKNtAKNSdRtLRDGkCGZlAEVAEDzRCzRUyNtMNDyNCyNDHXCHXSdXtLXxlYVYDyYCyYDzBHCzBHSzktQZjyWCyWDzTCzTDyAKCyAKDHRCHRSzTtQTDEWCEWDKMCKMDyPCyPSzLtQLDKYCKYDGTCGTUyktMZjEPCEPUdNtPNDENCENUEktAEZjGRCGRDEkCEZF1yWtMWSdLtLLUdYtPY]u]'
 m=[['nw','Z'],['eE','Y'],['eG','X'],['eK','W'],['tk','V'],['J1','U'],['eH','T'],['J2','S'],['ez','R'],['Bz','Q'],['Ad','P'],['ey','N'],['Ay','M'],['Bd','L'],['ah','K'],['xF','J'],['ai','H'],['ii','G'],['g[','F'],['hh','E'],['xj','D'],['te','C'],['m2','B'],['m1','A'],['ia','z'],['ha','y'],[']w','x'],['vp','w'],['u,','v'],[']}','u'],['sl','t'],['rO','s'],["q'",'r'],['nb','q'],['oI','p'],["{'",'o'],['k]','n'],['b[','m'],['jd','l'],['ed','k'],['gc','j'],[',2','i'],[',1','h'],['f[','g'],["':",'f'],['bc','e'],['aa','d'],['[0','c'],['],','b'],[',0','a']]
 for r in m:
  d=d.replace(r[1],r[0])
 d=eval(d)
 for k in d:
  if k['I']==g:
   g=k['O']
   return g

In [ ]:
verify_program(task_num, examples)

# Ｈ𝐀𝑷𝑷𝓎 🇰𝗮𝘨𝘨🇱𝖎Ｎɢ 💯